In [24]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [25]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [26]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [27]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp.solutions.holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [28]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Criando pasta para armazenar os dados

In [29]:
#para cada action são criadas pastas com subpastas index_video
def criarPastas(actions,numeros_videos_por_action):
    for action in actions: 
        for index_video in range(numeros_videos_por_action):
            try: 
                os.makedirs(os.path.join(DATA_PATH, action, str(index_video)))
            except:
                pass

In [30]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

#target que serão detectados
actions = np.array(['homem','obrigado', 'oi'])

# número de vídeos para cada target
numeros_videos_por_action = 3

criarPastas(actions,numeros_videos_por_action)


# Capturar vídeos

In [31]:
import cv2
import mediapipe as mp
import numpy as np

# Inicialize o MediaPipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

max_frames_per_video = 200
desired_fps = 30

In [32]:
def capturarVideos():
    for action in actions:
        print(action)
        for index_video in range(numeros_videos_por_action):
            sequence_str = str(index_video).zfill(1)
            video_path = f'./treino/{action}/{sequence_str}.mp4'

            cap = cv2.VideoCapture(video_path)

            if not cap.isOpened():
                print(f"Erro ao abrir o vídeo {video_path}.")
                continue

            holistic = mp_holistic.Holistic()

            frame_count = 0
            stop_processing = False  # Variável de controle

            while frame_count < max_frames_per_video and not stop_processing:
                ret, frame = cap.read()

                if not ret:
                    break

                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(rgb_frame)

                annotated_frame = rgb_frame.copy()

                draw_landmarks( annotated_frame, results)

                cv2.imshow('Video', cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR))

                frame_count += 1
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(index_video), str(frame_count))
                np.save(npy_path, keypoints)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    stop_processing = True  # Atualiza a variável de controle para encerrar
            while frame_count < max_frames_per_video:
                frame_count += 1
                npy_path = os.path.join(DATA_PATH, action, str(index_video), str(frame_count))
                np.save(npy_path, np.zeros_like(keypoints))
                
            cap.release()
            cv2.destroyAllWindows()

            if stop_processing:
                break  # Encerra os loops internos e avança para a próxima ação

In [20]:
def capturarVideos():
    for action in actions:
        for index_video in range(numeros_videos_por_action):
            sequence_str = str(index_video).zfill(1)
            video_path = f'./treino/{action}/{sequence_str}.mp4'

            cap = cv2.VideoCapture(video_path)

            if not cap.isOpened():
                print(f"Erro ao abrir o vídeo {video_path}.")
                continue

            holistic = mp_holistic.Holistic()

            frame_count = 0
            stop_processing = False  # Variável de controle

            while frame_count < max_frames_per_video and not stop_processing:
                ret, frame = cap.read()

                if not ret:
                    break

                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(rgb_frame)

                annotated_frame = rgb_frame.copy()

                draw_landmarks( annotated_frame, results)

                cv2.imshow('Video', cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR))

                frame_count += 1
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(index_video), str(frame_count))
                np.save(npy_path, keypoints)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    stop_processing = True  # Atualiza a variável de controle para encerrar

            cap.release()
            cv2.destroyAllWindows()

            if stop_processing:
                break  # Encerra os loops internos e avança para a próxima ação

In [33]:
capturarVideos()

homem
obrigado
oi
